In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # for printing progress bars

# The concept of criticality

## 1.1 Magnetization
Use the IsingModel class (from the `workshops` package) to simulate an Ising model for several values of temperature on a 20x25 lattice. You should create a two-dimensional lattice for the model using the `Grid.grid_2d` method.

Take the temperatures in interval [0.1, 4.0]. You have to specify the number of time steps (1000 or 2000 should be enough), and create a simple loop to aggregate the results.

Tip: IsingModel class has a simple method for simulation: simulate()

The magnetization at time *t* can be calculated as a average of all spins of the lattice:
$$M = \frac{1}{N}\sum_i \sigma_i $$
Then, average the magnetization over time and plot the averages versus temperature. You should get something looking like this:
![image](plots/ising_magnetization.png)

Tip: In case you are running out of time to run the simulation (it takes around 3 min on a laptop/colab), you can load simulation data `example_data/ising_dynamics.npz` with `np.load`.

In [ ]:
#### code here

## 1.2 Snapshots
To see in more detail what happens in the Ising system on a microscopic level. Plot two or three states of the lattice ("snapshots") for small temperature around 0.5, for large temperature ~4.0, and for temperature near the transition T = 2.3.

The results should look similarly to these:
![image](plots/ising_snap1.png)
![image](plots/ising_snap2.png)
![image](plots/ising_snap3.png)


In [2]:
#### code here

## 1.3 Binomial model (*)
Consider the Binomial Model, a variation of the Ising model where the spins are independent of each other and drawn from a specially crafted binomial distribution. 

Investigate the magnetization in this case. Can you spot differences between the Ising and Binomial models? What happened here, is our Binomial model critical?

To ensure you have a complete picture, show again the snapshots and describe any differences Binomial and Ising models.

Tip: In case you are running out of time to run the simulation (it takes around 3 min on a laptop/colab), you can load simulation data for the Binomial model `example_data/binomial_dynamics.npz` with `np.load`.

In [4]:
# a binomial model
from workshops import IsingModel

class BinomialModel(IsingModel):
    
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        
    def sweep(self,s: dict) -> None:
        p = 1/2*(self.calc_mag(T = self.T,J = self.J)+1)
        
        for n in s.keys():
            s[n] = -(2*np.random.binomial(n=1,p=p)-1)
        

In [ ]:
#### code here

# Towards a brain model



## 2.1 Haimovici model

Consider the Haimovici model. First, load a human connectome (adjacency matrix) `hagmann_connectome.npz` and *spy* on it to see how the connections are distributed (using `spy` function from `matplotlib.pyplot`). What do you see clearly?

Then, run Haimovici model simulation for `n_steps=2000` time steps (or more if you wish!). The threshold parameter should vary between [0.01,0.2].

The output matrix nodes can have 3 states:
* active (excited) nodes are represented by 1,
* refractory nodes are represented by -1,
* inactive (susceptible) nodes are represented by 0.

Plot a set of magnetizations for each sub-population of neurons using the function `plt.fill_between` (provide correct labels!) similar to this: 

![image](plots/brain_magnetization.png)





In [20]:
#### code here

## 2.2 Temperature or threshold? 

How is the threshold parameter of Haimovici model different from the temperature used in the Ising model?

Tip: Investigate temporal dynamics in all three sub-, critical- and super-critical parameter regimes.

Tip: Alternatively, you can plot sub-populations in the Ising model.

In [ ]:
#### code here

## 2.3 Artificial connectomes

What about artificial connectomes? Try Watts-Strogatz: create a function creates an artificial connectome based on Watts-Strogatz graph but with weights of connections from a real human connectome. The distribution of connections is approximately an exponential distribution with scale factor of $1/12.5$.

The Watts-Strogatz graphs can be easily created using `watts_strogatz_graph` from `networkx` library.
Create two *Watts-Strogatz connectomes* with 2000 nodes; one with mean number of neighbours 10, second with 2. The *rewiring* factor should be around 0.5.

Try plotting the fractions of active/refractory/inactive nodes creating a plot similar to one above.

Tip: If you run into problems with creating your own routine for the Watts-Strogatz connectome, you can use the function `workshops.watts_strogatz_connectome`.

In [21]:
#### code here

# Criticality in the Haimovici model



## 3.1 Own clustering algorithm (*)
Use `networkx` library to write a function that computes the size of the largest and second-largest clusters in a system. The cluster is defined as a set of active nodes which are connected via the connectome.

Tip: use `networkx`'s function `connected_components` which finds all connected components of a graph.

In [28]:
#### code here

## 3.2 Clusters in the Ising model
Take the Ising model for low temperature, e.g. $T=1.5$ (remember about the magnetization plot from the first exercise), and plot the snapshots of the lattice spins. What are the sizes of the largest cluster and second largest clusters?

What about $T = T_c$?


In [27]:
#### code here

## 3.3 Criticality indicators in the Haimovici model

Find the cluster sizes in the Haimovici model. Use a predefined function `workshops.batch_clusters`. Try plotting the average size of the largest cluster vs. threshold. What is the difference between the largest cluster and the second largest cluster?

Find the standard deviation of the total activity. Next prepare a function which computes the autocorrelation coefficient at lat $\tau=1$. Do these quantities have something in common? What about the clusters?

In [26]:
#### code here

## 3.5 Detective work
Load file with activities, thresholds, and computed clusters. Can you say whether the system is critical?

In [164]:
#### code here

# Playing around



## 4.1 Lobotomy
Try to disconnect the hemispheres in the human connectome. Note that left hemisphere is represented by nodes from 0 to 498, right hemisphere by nodes 499 to 998.

`Spy` on the adjacency matrix to check the difference. Then, using the connectome with disconnected hemispheres run simulations and check the criticality using clusters and other methods (e.g. std. dev. of activity).

In [ ]:
#### code here

## 4.2 Stroke
Artificial Stroke: remove connections between Sensory-Motor RSN (labelled by 4) and the rest of the brain. Check the criticality of the system using activity measure, e.g. standard deviation of the total activity, and compare with result computed using second largest cluster size.


Try also other labels:

VisM (Medial Visual) - 1
VisL (Lateral Visual) - 2
Aud (Auditory) - 3
SM (Sensory-Motor) - 4
DMN (Default Mode Network) - 5
EC (Executive Control) - 6
DorL (Dorsal Visual Stream Left) - 7
DorR (Dorsal Visual Stream Right) - 8




In [ ]:
#### code here

## 4.3 Epilepsy
Enchance connectome connections by a constant value. Find the criticality using different measures and compare them with the healthy connectome.

In [29]:
#### code here